
# Preprocessing & Modelling – Introduction

- In this notebook, we take the insights from the EDA and transform the raw dataset into a machine-ready format.
- The goal is to build a clean preprocessing pipeline and train a predictive model for the binary classification target at_risk, which identifies employees with low satisfaction (rating ≤ 3).

- We will:

1. Prepare the data for modelling

    - encode categorical variables,

    - handle missing values,

    - select relevant features,

    - build a reproducible scikit-learn pipeline.

2. Train and evaluate classification models

    - a baseline model,

    - a simple linear model,

    - and a Random Forest, which is well-suited to our encoded categorical data.

This notebook follows a structured ML workflow to ensure reproducibility, clarity, and alignment with the business goal:

**predict which employees are at risk of low satisfaction.**




### Presentation Link

[Redirect to presentation ](https://shorturl.at/2lxwv)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Load the cleaned dataset
df_encoded = pd.read_csv("/content/pwc_with_at_risk.csv")


In [ ]:
df_encoded

,date,rating,country,employee_status,position_level,function_area,recommender_bin,perspective_commerciale_bin,approbation_pdg_bin,year,at_risk
0,2022-04-19,2.0,Australia,0,NaN,Other,-1.0,NaN,NaN,2022,1
1,2021-12-29,3.0,Pakistan,0,Associate,Consulting–Advisory,1.0,-1.0,1.0,2021,1
2,2022-03-29,4.0,Singapore,1,Director,Other,NaN,NaN,NaN,2022,0
3,2022-04-12,4.0,Ireland,1,Senior Manager,Tech–IT–Data,NaN,NaN,NaN,2022,0
4,2022-04-12,5.0,Malaysia,1,Senior Manager,Other,NaN,NaN,NaN,2022,0
...,...,...,...,...,...,...,...,...,...,...,...
43447,2023-02-20,4.0,United States,0,Associate,Tech–IT–Data,NaN,NaN,NaN,2023,0
43448,2023-02-21,5.0,Indonesia,0,Associate,Audit–Assurance,1.0,1.0,1.0,2023,0
43449,2023-02-21,4.0,Singapore,1,Associate,Other,NaN,NaN,NaN,2023,0
43450,2023-02-28,3.0,United States,1,Senior Associate,Other,NaN,NaN,NaN,2023,1


 # 1. Preprocessing

## A. Ordinal encoding on position_level

In [ ]:
position_order = ['Intern', 'Entry Level', 'Associate', 'Senior Associate', 'Manager', 'Senior Manager', 'Director', 'Partner', 'Unknown']

# Fill NaN values with 'Unknown' BEFORE encoding
df_encoded['position_level'] = df_encoded['position_level'].fillna('Unknown')

# Apply ordinal encoding
df_encoded['position_level_encoded'] = pd.Categorical(df_encoded['position_level'], categories=position_order, ordered=True).codes

print("DataFrame after ordinal encoding 'position_level' column:")
display(df_encoded.head())

DataFrame after ordinal encoding 'position_level' column:


,date,rating,country,employee_status,position_level,function_area,recommender_bin,perspective_commerciale_bin,approbation_pdg_bin,year,at_risk,position_level_encoded
0,2022-04-19,2.0,Australia,0,Unknown,Other,-1.0,NaN,NaN,2022,1,8
1,2021-12-29,3.0,Pakistan,0,Associate,Consulting–Advisory,1.0,-1.0,1.0,2021,1,2
2,2022-03-29,4.0,Singapore,1,Director,Other,NaN,NaN,NaN,2022,0,6
3,2022-04-12,4.0,Ireland,1,Senior Manager,Tech–IT–Data,NaN,NaN,NaN,2022,0,5
4,2022-04-12,5.0,Malaysia,1,Senior Manager,Other,NaN,NaN,NaN,2022,0,5


## B. One Hot encoding on function_area column:

In [ ]:
# Reload the cleaned dataset to ensure 'function_area' is present
df_encoded = pd.read_csv("/content/pwc_with_at_risk.csv")

# Re-apply ordinal encoding for 'position_level' as it was done before
position_order = ['Intern', 'Entry Level', 'Associate', 'Senior Associate', 'Manager', 'Senior Manager', 'Director', 'Partner', 'Unknown']
df_encoded['position_level'] = df_encoded['position_level'].fillna('Unknown')
df_encoded['position_level_encoded'] = pd.Categorical(df_encoded['position_level'], categories=position_order, ordered=True).codes

# Apply get_dummies to df_encoded for 'function_area' and convert new columns to int type.
df_encoded = pd.get_dummies(df_encoded, columns=['function_area'], prefix='function_area', drop_first=False, dtype=int)

# Display the head of the DataFrame to show the new one-hot encoded columns
print("DataFrame after one-hot encoding 'function_area' column:")
display(df_encoded.head())

DataFrame after one-hot encoding 'function_area' column:


,date,rating,country,employee_status,position_level,recommender_bin,perspective_commerciale_bin,approbation_pdg_bin,year,at_risk,position_level_encoded,function_area_Audit–Assurance,function_area_Consulting–Advisory,function_area_Deals–Transactions,function_area_Other,function_area_Risk,function_area_Support (HR/Ops/Finance),function_area_Tax,function_area_Tech–IT–Data
0,2022-04-19,2.0,Australia,0,Unknown,-1.0,NaN,NaN,2022,1,8,0,0,0,1,0,0,0,0
1,2021-12-29,3.0,Pakistan,0,Associate,1.0,-1.0,1.0,2021,1,2,0,1,0,0,0,0,0,0
2,2022-03-29,4.0,Singapore,1,Director,NaN,NaN,NaN,2022,0,6,0,0,0,1,0,0,0,0
3,2022-04-12,4.0,Ireland,1,Senior Manager,NaN,NaN,NaN,2022,0,5,0,0,0,0,0,0,0,1
4,2022-04-12,5.0,Malaysia,1,Senior Manager,NaN,NaN,NaN,2022,0,5,0,0,0,1,0,0,0,0


## B/ Encoding the country column

The country variable has 65 unique values, so One-Hot Encoding would create 65 extra columns : too large and inefficient for this project.
Label Encoding keeps the dataset compact by assigning one integer per country.

This choice works because the model used later is a RandomForestClassifier, a tree-based model that does not assume any numeric order between encoded labels.
Trees simply split on thresholds, so the numeric values do not create bias.

**Label Encoding is the most practical and compatible option given the high cardinality and the model we use.**

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder instance
label_encoder = LabelEncoder()

# Apply Label Encoding to the 'country' column
df_encoded['country_encoded'] = label_encoder.fit_transform(df_encoded['country'])

# Display the head of the DataFrame to show the new encoded column
print("DataFrame after label encoding 'country' column:")
display(df_encoded.head())

# Explicitly show columns to confirm 'country_encoded' is present
print("Columns in df_encoded after country encoding:", df_encoded.columns.tolist())

# You can also see the mapping of countries to their encoded values
print("\nMapping of countries to their encoded values:")
for i, country in enumerate(label_encoder.classes_):
    print(f"{country}: {i}")

DataFrame after label encoding 'country' column:


,date,rating,country,employee_status,position_level,recommender_bin,perspective_commerciale_bin,approbation_pdg_bin,year,at_risk,position_level_encoded,function_area_Audit–Assurance,function_area_Consulting–Advisory,function_area_Deals–Transactions,function_area_Other,function_area_Risk,function_area_Support (HR/Ops/Finance),function_area_Tax,function_area_Tech–IT–Data,country_encoded
0,2022-04-19,2.0,Australia,0,Unknown,-1.0,NaN,NaN,2022,1,8,0,0,0,1,0,0,0,0,1
1,2021-12-29,3.0,Pakistan,0,Associate,1.0,-1.0,1.0,2021,1,2,0,1,0,0,0,0,0,0,42
2,2022-03-29,4.0,Singapore,1,Director,NaN,NaN,NaN,2022,0,6,0,0,0,1,0,0,0,0,49
3,2022-04-12,4.0,Ireland,1,Senior Manager,NaN,NaN,NaN,2022,0,5,0,0,0,0,0,0,0,1,24
4,2022-04-12,5.0,Malaysia,1,Senior Manager,NaN,NaN,NaN,2022,0,5,0,0,0,1,0,0,0,0,34


Columns in df_encoded after country encoding: ['date', 'rating', 'country', 'employee_status', 'position_level', 'recommender_bin', 'perspective_commerciale_bin', 'approbation_pdg_bin', 'year', 'at_risk', 'position_level_encoded', 'function_area_Audit–Assurance', 'function_area_Consulting–Advisory', 'function_area_Deals–Transactions', 'function_area_Other', 'function_area_Risk', 'function_area_Support (HR/Ops/Finance)', 'function_area_Tax', 'function_area_Tech–IT–Data', 'country_encoded']

Mapping of countries to their encoded values:
Argentina: 0
Australia: 1
Bahrain: 2
Brazil: 3
Canada: 4
Chile: 5
China: 6
Colombia: 7
Cyprus: 8
Czechia: 9
Denmark: 10
Egypt: 11
Estonia: 12
Finland: 13
France: 14
Georgia: 15
Germany: 16
Ghana: 17
Gibraltar: 18
Greece: 19
Hong Kong: 20
Hungary: 21
India: 22
Indonesia: 23
Ireland: 24
Israel: 25
Italy: 26
Japan: 27
Jordan: 28
Kazakhstan: 29
Kenya: 30
Lebanon: 31
Libya: 32
Luxembourg: 33
Malaysia: 34
Mauritius: 35
Mexico: 36
Netherlands: 37
New Zealand: 38



## C. Imputation of missing values

Three columns in our dataset : recommender_bin  approbation_pdg_bin and perspective_commerciale are the columns with the most missing values. In order to make future predictions, we must find a way to impute these missing values.

recommender_bin column is a collection of -1(Negtive) and +1(Positive) remarks by people who recommend or do not recommend the company for work. Similarly, approbation_pdg_bin is if employees recommend the direction of the company and prespective_commerciale is if employees think the company is a good commercial perspective in business.

The challenge in imputing these values is that recommender_bin column has only -1 and +1 as recommendations so only Yes and No, while in the other two columns we also have 0 along with -1 and 1. This means that we need to impute these three columns with different parameters to keep the original format of having only -1 and 1 imputed for the recommender_bin column while -1, 0 and 1 for the rest of the two. It is also important to keep the distribution of -1, 0 and 1 in the columns the same after imputation.

In [ ]:
columns_with_missing = ['recommender_bin', 'approbation_pdg_bin', 'perspective_commerciale_bin']

for col in columns_with_missing:
    print(f"\nProportion of values in '{col}':")
    percentages = df_encoded[col].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'
    display(percentages.sort_index())


Proportion of values in 'recommender_bin':


,proportion
recommender_bin,
-1.0,22.28%
1.0,77.72%



Proportion of values in 'approbation_pdg_bin':


,proportion
approbation_pdg_bin,
-1.0,6.67%
0.0,33.77%
1.0,59.56%



Proportion of values in 'perspective_commerciale_bin':


,proportion
perspective_commerciale_bin,
-1.0,10.0%
0.0,22.68%
1.0,67.32%


For imputating missing values, we also kept into account the correlation of these three columns with other columns in the dataset. In the correlation matrix explined in the EDA part,

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Columns
cols_all = ["recommender_bin", "approbation_pdg_bin", "perspective_commerciale_bin"]

# Step 1: Track missing values before imputation
for c in cols_all:
    df_encoded[f"{c}_was_missing"] = df_encoded[c].isna()

# Dictionary to store evaluation results
eval_results = {}

# Step 2: Impute recommender_bin with only -1 or 1
target = "recommender_bin"
train_mask = df_encoded[target].notna()

# Simulate missingness for evaluation
np.random.seed(42)
eval_mask = train_mask.copy()
# Sample only from True values to set to False for evaluation
true_indices = eval_mask[eval_mask].index
sampled_indices = np.random.choice(true_indices, size=int(len(true_indices) * 0.2), replace=False)
eval_mask.loc[sampled_indices] = False

# Define feature columns for recommender_bin imputation
feature_cols_recommender = ["rating", "approbation_pdg_bin", "perspective_commerciale_bin"]

# Prepare X_train, y_train, X_test, y_test
X_train = df_encoded.loc[eval_mask, feature_cols_recommender].fillna(0) # Fill NaNs in features
y_train = df_encoded.loc[eval_mask, target]

# For testing, ensure y_test only contains known values from the original non-NaN entries
test_indices_known = (~eval_mask) & train_mask
X_test = df_encoded.loc[test_indices_known, feature_cols_recommender].fillna(0) # Fill NaNs in features
y_test = df_encoded.loc[test_indices_known, target]

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_pred = np.where(y_pred == 0, np.random.choice([-1, 1], size=len(y_pred)), y_pred)

# Evaluation
eval_results[target] = {
    "accuracy": accuracy_score(y_test, y_pred),
    "f1_macro": f1_score(y_test, y_pred, average="macro"),
    "confusion_matrix": confusion_matrix(y_test, y_pred, labels=[-1, 1])
}

# Impute real missing values
miss_mask = df_encoded[target].isna()
if miss_mask.any():
    X_miss = df_encoded.loc[miss_mask, feature_cols_recommender].fillna(0) # Fill NaNs in features for prediction
    preds = clf.predict(X_miss)
    preds = np.where(preds == 0, np.random.choice([-1, 1], size=len(preds)), preds)
    df_encoded.loc[miss_mask, target] = preds

df_encoded[target] = df_encoded[target].astype(int)

# Step 3: Impute approbation_pdg_bin and perspective_commerciale_bin with -1, 0, or 1
for target in ["approbation_pdg_bin", "perspective_commerciale_bin"]:
    train_mask = df_encoded[target].notna()

    np.random.seed(42)
    eval_mask = train_mask.copy()
    # Sample only from True values to set to False for evaluation
    true_indices = eval_mask[eval_mask].index
    sampled_indices = np.random.choice(true_indices, size=int(len(true_indices) * 0.2), replace=False)
    eval_mask.loc[sampled_indices] = False

    # Define feature columns for current target imputation
    feature_cols_other = ["rating", "recommender_bin"] + [c for c in cols_all if c != target and c != "recommender_bin"]

    # Prepare X_train, y_train, X_test, y_test
    X_train = df_encoded.loc[eval_mask, feature_cols_other].fillna(0) # Fill NaNs in features
    y_train = df_encoded.loc[eval_mask, target]

    # For testing, ensure y_test only contains known values from the original non-NaN entries
    test_indices_known = (~eval_mask) & train_mask
    X_test = df_encoded.loc[test_indices_known, feature_cols_other].fillna(0) # Fill NaNs in features
    y_test = df_encoded.loc[test_indices_known, target]

    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    y_pred = np.clip(y_pred, -1, 1)

    # Evaluation
    eval_results[target] = {
        "accuracy": accuracy_score(y_test, y_pred),
        "f1_macro": f1_score(y_test, y_pred, average="macro"),
        "confusion_matrix": confusion_matrix(y_test, y_pred, labels=[-1, 0, 1])
    }

    # Impute real missing values
    miss_mask = df_encoded[target].isna()
    if miss_mask.any():
        X_miss = df_encoded.loc[miss_mask, feature_cols_other].fillna(0) # Fill NaNs in features for prediction
        preds = clf.predict(X_miss)
        preds = np.clip(preds, -1, 1)
        df_encoded.loc[miss_mask, target] = preds

    df_encoded[target] = df_encoded[target].astype(int)

# Step 4: Sanity check
for c in cols_all:
    unique_vals = df_encoded[c].unique()
    print(f"\n✅ {c} unique values after imputation: {unique_vals}")

# Step 5: Print evaluation results
for col, res in eval_results.items():
    print(f"\n📊 Evaluation for {col}:")
    print(f"Accuracy: {res['accuracy']:.3f}")
    print(f"F1 Score (macro): {res['f1_macro']:.3f}")
    print("Confusion Matrix:\n", pd.DataFrame(res["confusion_matrix"],
          index=["-1", "0", "1"][:res["confusion_matrix"].shape[0]],
          columns=["Pred -1", "Pred 0", "Pred 1"][:res["confusion_matrix"].shape[1]]))



✅ recommender_bin unique values after imputation: [-1  1]

✅ approbation_pdg_bin unique values after imputation: [ 0  1 -1]

✅ perspective_commerciale_bin unique values after imputation: [ 0 -1  1]

📊 Evaluation for recommender_bin:
Accuracy: 0.891
F1 Score (macro): 0.818
Confusion Matrix:
     Pred -1  Pred 0
-1      456     328
0        57    2704

📊 Evaluation for approbation_pdg_bin:
Accuracy: 0.687
F1 Score (macro): 0.597
Confusion Matrix:
     Pred -1  Pred 0  Pred 1
-1      108     127      34
0        66     755     579
1        30     452    1965

📊 Evaluation for perspective_commerciale_bin:
Accuracy: 0.784
F1 Score (macro): 0.697
Confusion Matrix:
     Pred -1  Pred 0  Pred 1
-1      227     149      91
0        37     724     305
1        15     413    2707


In [ ]:
imputed_columns = ['recommender_bin', 'approbation_pdg_bin', 'perspective_commerciale_bin']

for col in imputed_columns:
    print(f"\nProportion of values in '{col}':")
    # Calculate proportions and format as percentages
    percentages = df_encoded[col].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'
    display(percentages.sort_index())


Proportion of values in 'recommender_bin':


,proportion
recommender_bin,
-1,14.74%
1,85.26%



Proportion of values in 'approbation_pdg_bin':


,proportion
approbation_pdg_bin,
-1,3.91%
0,47.77%
1,48.32%



Proportion of values in 'perspective_commerciale_bin':


,proportion
perspective_commerciale_bin,
-1,5.76%
0,43.09%
1,51.16%


In [ ]:
df_encoded['position_level']

,position_level
0,Unknown
1,Associate
2,Director
3,Senior Manager
4,Senior Manager
...,...
43447,Associate
43448,Associate
43449,Associate
43450,Senior Associate


# 2. Machine learning model

## 1.  RandomForestClassifier (Baseline - no SMOTE)

### Feature Selection and Model Training

Now, we'll prepare our dataset for modeling by separating features (X) from the target variable (y). The target variable `at_risk` identifies employees with low satisfaction.

We will then split the data into training and testing sets to evaluate our model's performance on unseen data. Finally, we'll train a `RandomForestClassifier`.

In [ ]:
# Define features (X) and target (y)
X = df_encoded.drop(columns=['date', 'rating','country','position_level','recommender_bin_was_missing','approbation_pdg_bin_was_missing','perspective_commerciale_bin_was_missing','at_risk'])
y = df_encoded['at_risk']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

display(X)

Features shape: (43452, 15)
Target shape: (43452,)


,employee_status,recommender_bin,perspective_commerciale_bin,approbation_pdg_bin,year,position_level_encoded,function_area_Audit–Assurance,function_area_Consulting–Advisory,function_area_Deals–Transactions,function_area_Other,function_area_Risk,function_area_Support (HR/Ops/Finance),function_area_Tax,function_area_Tech–IT–Data,country_encoded
0,0,-1,0,0,2022,8,0,0,0,1,0,0,0,0,1
1,0,1,-1,1,2021,2,0,1,0,0,0,0,0,0,42
2,1,1,0,0,2022,6,0,0,0,1,0,0,0,0,49
3,1,1,0,0,2022,5,0,0,0,0,0,0,0,1,24
4,1,1,1,1,2022,5,0,0,0,1,0,0,0,0,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43447,0,1,0,0,2023,2,0,0,0,0,0,0,0,1,62
43448,0,1,1,1,2023,2,1,0,0,0,0,0,0,0,23
43449,1,1,0,0,2023,2,0,0,0,1,0,0,0,0,49
43450,1,1,0,0,2023,3,0,0,0,1,0,0,0,0,62


### Splitting Data into Training and Testing Sets

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (34761, 15)
X_test shape: (8691, 15)
y_train shape: (34761,)
y_test shape: (8691,)


### Training the RandomForestClassifier

We will now initialize and train a `RandomForestClassifier` on our training data. After training, we'll use the model to make predictions on the test set and evaluate its performance using a classification report and confusion matrix.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

# Train the model
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
print("\nAccuracy Score:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy Score: 0.7518122195374526

Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.80      0.82      5920
           1       0.60      0.64      0.62      2771

    accuracy                           0.75      8691
   macro avg       0.72      0.72      0.72      8691
weighted avg       0.76      0.75      0.75      8691


Confusion Matrix:
 [[4754 1166]
 [ 991 1780]]


- Why it's 'not really working' for identifying at-risk employees:

    - High Number of False Negatives (FN): Your primary goal is to predict which employees are at risk. The model is failing to identify 1245 out of 2771 actual at-risk employees (recall of 0.55). This means almost half of the employees who genuinely need attention are being missed. In a business context where you want to intervene to prevent low satisfaction, missing these individuals is a significant issue.

    - Low Recall for the 'at-risk' class (0.55): This is the most concerning metric. It tells you that out of all the employees who are truly at-risk, your model is only catching about 55% of them. This makes the model unreliable for proactively identifying and supporting these employees.

    - Low Precision for the 'at-risk' class (0.56): When the model does predict someone is 'at-risk', it's only correct about 56% of the time. This means nearly half of your interventions based on these predictions would be directed at employees who aren't actually at-risk (False Positives: 1209). This can lead to wasted resources and potentially alienate employees who are falsely identified.

    - Imbalance Impact: The model is biased towards the majority class (not at-risk), where it achieves much higher precision and recall. The class_weight='balanced' parameter helps, but doesn't fully overcome the inherent difficulty of predicting the minority class.

    - In essence, while the overall accuracy is okay, the model struggles specifically with the at_risk class, exhibiting a high rate of both false positives and, more critically, false negatives. This suggests that the model isn't effectively learning the patterns that distinguish at-risk employees from others.





## 2. RandomForestClassifier (with SMOTE )


Apply SMOTE (Synthetic Minority Over-sampling Technique) to the training data (`X_train`, `y_train`) to balance the classes and store the resampled data in `X_train_smote` and `y_train_smote`.

###  Apply SMOTE to Training Data



Before applying SMOTE, the `imblearn` library needs to be installed. This step ensures that the necessary `SMOTE` class is available for import and use.



In [ ]:
pip install imblearn


Now that `imblearn` is installed, I will import the `SMOTE` class, initialize it, and apply it to the training data (`X_train`, `y_train`) to balance the classes, storing the results in `X_train_smote` and `y_train_smote`.



In [ ]:
from imblearn.over_sampling import SMOTE

# Initialize SMOTE
smote = SMOTE(random_state=42)

# Apply SMOTE to the training data
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print(f"Original training set shape: {X_train.shape}, {y_train.shape}")
print(f"Resampled training set shape: {X_train_smote.shape}, {y_train_smote.shape}")

print("\nClass distribution before SMOTE:\n", y_train.value_counts())
print("\nClass distribution after SMOTE:\n", y_train_smote.value_counts())

Original training set shape: (34761, 15), (34761,)
Resampled training set shape: (47360, 15), (47360,)

Class distribution before SMOTE:
 at_risk
0    23680
1    11081
Name: count, dtype: int64

Class distribution after SMOTE:
 at_risk
1    23680
0    23680
Name: count, dtype: int64


### Retrain RandomForestClassifier with Balanced Data



Initialize a RandomForestClassifier with the specified parameters, train it on the SMOTE-resampled data, and then make predictions on the original test set.



In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the Random Forest Classifier with balanced class weights
rf_classifier_smote = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

# Train the model on the SMOTE-resampled data
rf_classifier_smote.fit(X_train_smote, y_train_smote)

# Make predictions on the original test set
y_pred_smote = rf_classifier_smote.predict(X_test)

print("RandomForestClassifier trained on SMOTE data and predictions made on X_test.")

RandomForestClassifier trained on SMOTE data and predictions made on X_test.



The previous step trained a RandomForestClassifier on SMOTE-resampled data. The next logical step is to evaluate its performance using classification metrics to see if SMOTE improved the model's ability to identify at-risk employees.



In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Evaluate the model trained with SMOTE
print("\nEvaluation of RandomForestClassifier trained with SMOTE:")
print("Accuracy Score:", accuracy_score(y_test, y_pred_smote))
print("\nClassification Report:\n", classification_report(y_test, y_pred_smote))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_smote))


Evaluation of RandomForestClassifier trained with SMOTE:
Accuracy Score: 0.7460591416407778

Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.78      0.81      5920
           1       0.59      0.67      0.63      2771

    accuracy                           0.75      8691
   macro avg       0.71      0.73      0.72      8691
weighted avg       0.76      0.75      0.75      8691


Confusion Matrix:
 [[4628 1292]
 [ 915 1856]]


### Analysis of Model Performance with SMOTE

After applying SMOTE to balance the training data, the `RandomForestClassifier` was retrained and evaluated. Here's a comparison with the previous model (without SMOTE):

**Key Observations:**

*   **Overall Accuracy:** The overall accuracy decreased slightly from 0.68 to 0.67. This is expected when trying to improve performance on the minority class, as the model might make more errors on the majority class.
*   **Recall for 'at-risk' class (Class 1):** Improved from 0.59 to 0.61. This is a positive outcome, as it means the model is now better at identifying actual at-risk employees, reducing the number of false negatives (missed at-risk individuals). The number of false negatives decreased from 1145 to 1094.
*   **Precision for 'at-risk' class (Class 1):** Decreased slightly from 0.50 to 0.49. This indicates that when the model predicts someone is 'at-risk', it is slightly less often correct. The number of false positives (employees incorrectly identified as at-risk) increased from 1617 to 1769.
*   **F1-Score for 'at-risk' class (Class 1):** Remained relatively stable at 0.54. The F1-score is the harmonic mean of precision and recall, suggesting that while there was a trade-off, the overall balance between precision and recall for the minority class didn't drastically change.

**Conclusion:**

SMOTE successfully improved the model's ability to identify the minority 'at-risk' class by increasing recall and reducing false negatives. However, this came at the cost of a slight decrease in overall accuracy and an increase in false positives. The choice of whether this trade-off is acceptable depends on the specific business context: is it more critical to identify as many at-risk employees as possible (prioritizing recall) even if it means some false alarms, or to be highly precise in predictions (prioritizing precision) even if some at-risk employees are missed?

This demonstrates that addressing class imbalance often involves navigating trade-offs, and further tuning or alternative strategies might be explored to achieve a more optimal balance.

## 3. Train LightGBM Classifier

Initialize and train a LightGBM Classifier on the SMOTE-resampled training data (`X_train_smote`, `y_train_smote`). We will include `class_weight='balanced'` in the LightGBM model, even with SMOTE, as it can further help the model focus on the minority class during training.



To initialize and train the LightGBM Classifier as per the instructions, I need to import the LGBMClassifier, create an instance with the specified parameters, and then fit it to the SMOTE-resampled training data.



In [ ]:
from lightgbm import LGBMClassifier

# Initialize the LightGBM Classifier
lgbm_classifier = LGBMClassifier(class_weight='balanced', random_state=42)

# Train the model on the SMOTE-resampled data
lgbm_classifier.fit(X_train_smote, y_train_smote)

print("LightGBM Classifier trained successfully on SMOTE-resampled data.")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 23680, number of negative: 23680
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007081 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 118
[LightGBM] [Info] Number of data points in the train set: 47360, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
LightGBM Classifier trained successfully on SMOTE-resampled data.



The next step is to make predictions on the original test set using the newly trained LightGBM model.



In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score

# Make predictions on the original test set
y_pred_lgbm = lgbm_classifier.predict(X_test)

print("Predictions made on X_test using LightGBM Classifier.")

Predictions made on X_test using LightGBM Classifier.



Now that predictions have been made, the next step is to evaluate the LightGBM model's performance using classification metrics and ROC AUC score



In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score

# Evaluate the LightGBM model
print("\nEvaluation of LightGBM Classifier trained with SMOTE:")
print("Accuracy Score:", accuracy_score(y_test, y_pred_lgbm))
print("\nClassification Report:\n", classification_report(y_test, y_pred_lgbm))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_lgbm))
print("\nROC AUC Score:", roc_auc_score(y_test, lgbm_classifier.predict_proba(X_test)[:, 1]))


Evaluation of LightGBM Classifier trained with SMOTE:
Accuracy Score: 0.7546887584857899

Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.76      0.81      5920
           1       0.59      0.73      0.66      2771

    accuracy                           0.75      8691
   macro avg       0.73      0.75      0.73      8691
weighted avg       0.78      0.75      0.76      8691


Confusion Matrix:
 [[4523 1397]
 [ 735 2036]]

ROC AUC Score: 0.8383758363650551


In [ ]:
import pandas as pd

# Define the metrics for each model based on previous outputs
model_comparison_data = {
    'Model': [
        'RandomForest (No SMOTE)',
        'RandomForest (with SMOTE)',
        'LightGBM (with SMOTE)'
    ],
    'Overall Accuracy': [
        0.7518, # from cell ca1d8930
        0.7461, # from cell cb64e96c
        0.7547  # from cell ba135151
    ],
    'Recall (Class 1)': [
        0.64, # from cell ca1d8930
        0.67, # from cell cb64e96c
        0.73  # from cell ba135151
    ],
    'Precision (Class 1)': [
        0.60, # from cell ca1d8930
        0.59, # from cell cb64e96c
        0.59  # from cell ba135151
    ],
    'F1-Score (Class 1)': [
        0.62, # from cell ca1d8930
        0.63, # from cell cb64e96c
        0.66  # from cell ba135151
    ],
    'ROC AUC Score': [
        'N/A', # Not calculated for the first RF model
        'N/A', # Not calculated for the second RF model
        0.8384 # from cell ba135151
    ]
}

# Create the DataFrame
model_comparison_df = pd.DataFrame(model_comparison_data)

print("Model Comparison Table:")
display(model_comparison_df)

Model Comparison Table:


,Model,Overall Accuracy,Recall (Class 1),Precision (Class 1),F1-Score (Class 1),ROC AUC Score
0,RandomForest (No SMOTE),0.7518,0.64,0.60,0.62,N/A
1,RandomForest (with SMOTE),0.7461,0.67,0.59,0.63,N/A
2,LightGBM (with SMOTE),0.7547,0.73,0.59,0.66,0.8384


#Analysis and Conclusion

Upon comparing the three models, the **LightGBM Classifier trained with SMOTE** stands out as the best performer for our objective of identifying at-risk employees.

Here's why:

*   **Highest Recall for 'at-risk' class (0.73):** This is the most critical metric for our problem. LightGBM with SMOTE correctly identifies 73% of actual at-risk employees, which is significantly better than the RandomForest models (0.64 and 0.67). This means fewer at-risk employees are missed, allowing for more targeted interventions.
*   **Highest F1-Score for 'at-risk' class (0.66):** The F1-score balances precision and recall. LightGBM's higher F1-score indicates a better overall balance in identifying the minority class compared to the other models.
*   **Highest Overall Accuracy (0.7547):** While not the primary focus, LightGBM also achieves a slightly higher overall accuracy, indicating good performance across both classes.
*   **Strong ROC AUC Score (0.8384):** The ROC AUC score measures the model's ability to distinguish between classes. A score of 0.8384 is good, suggesting that the model is robust in differentiating between at-risk and not-at-risk employees.
*   **Effectiveness of SMOTE:** For both RandomForest and LightGBM, applying SMOTE was beneficial, particularly in improving recall for the minority class, demonstrating its importance in handling imbalanced datasets.

**In conclusion, the LightGBM Classifier, when combined with SMOTE for handling class imbalance, provides the most effective solution for predicting employees at risk of low satisfaction. Its superior recall for the 'at-risk' class ensures that the organization can proactively identify and support a larger proportion of employees who genuinely need attention, aligning well with the business goal.**